In [1]:
"""
This code is based on Facebook's HDemucs code: https://github.com/facebookresearch/demucs
"""
import logging
import os
import sys
import torch
from pathlib import Path
import hydra
import wandb
import torchaudio
import IPython.display as ipd


cwd = Path().resolve()
prj_dir = os.path.dirname(os.path.dirname(os.path.abspath(cwd)))
print(f'prj_dir:{prj_dir}')
sys.path.append(prj_dir)

from src.data.datasets import LrHrSet
from src.ddp import distrib
from src.evaluate import evaluate
from src.models import modelFactory
from src.utils import bold
from src.wandb_logger import _init_wandb_run
from hydra import initialize, compose

logger = logging.getLogger(__name__)

SERIALIZE_KEY_MODELS = 'models'
SERIALIZE_KEY_BEST_STATES = 'best_states'
SERIALIZE_KEY_STATE = 'state'


initialize('../../conf') # Assume the configuration file is in the current folder
args = compose(config_name='main_config')

prj_dir:/data1/hjh/pycharm_projects/othergithub_prjs/super_resolution/aero


/home/huangjiahong.dracu/miniconda2/envs/aero/lib/python3.8/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'main_config': Defaults list is missing `_self_`. See https://hydra.cc/docs/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)


# load model

In [4]:
def _load_model(args):
    model_name = args.experiment.model
    checkpoint_file = Path(args.checkpoint_file)
    model = modelFactory.get_model(args)['generator']
    package = torch.load(checkpoint_file, 'cpu')
    load_best = args.continue_best
    if load_best:
        logger.info(bold(f'Loading model {model_name} from best state.'))
        model.load_state_dict(
            package[SERIALIZE_KEY_BEST_STATES][SERIALIZE_KEY_MODELS]['generator'][SERIALIZE_KEY_STATE])
    else:
        logger.info(bold(f'Loading model {model_name} from last state.'))
        model.load_state_dict(package[SERIALIZE_KEY_MODELS]['generator'][SERIALIZE_KEY_STATE])

    return model.cuda()

args.checkpoint_file="/mnt/cephfs/hjh/train_record/super_resolution/aero/train_outputs/16-24/aero-nfft=512-hl=256/checkpoint.th"

model=_load_model(args)

print("lond model done!")

lond model done!


In [10]:
# x = torch.randn(size=(1, 1, 16000)).float().cuda()
file="/mnt/cephfs/hjh/train_record/super_resolution/aero/dataset/vctk/rs_wav16k/p314/p314_412.wav"
x, sr = torchaudio.load(file) #音频sr=16000
print("原始音频:")
ipd.display(ipd.Audio(x, rate=sr))
x=x.unsqueeze(0).cuda()#[1,1,number]
y=model(x)

print("超分后音频:")
print(y.size())
ipd.display(ipd.Audio(y.squeeze().detach().cpu(), rate=24000))

原始音频:


超分后音频:
torch.Size([1, 1, 67666])
